In [8]:
import bempp.api
import numpy as np

from bempp.api.operators.boundary.maxwell import multitrace_operator
from bempp.api.operators.potential.maxwell import electric_field

k = 1
mu = 1
element_size = 0.5
length_cavity = 1



grid = bempp.api.shapes.cube(
    length=length_cavity, h=element_size,
    origin=(-length_cavity/2, -length_cavity/2, -length_cavity/2)
)
    
def inc(x):
    return np.array([np.exp(1j * k * x[2]), 0. * x[2], 0. * x[2]])

def tang(x, n, domain_ind, result):
    result[:] = np.cross(inc(x), n, axis=0)

multitrace = multitrace_operator(grid, k)
bc_space = multitrace.range_spaces[1]
snc_space = multitrace.dual_to_range_spaces[1]

grid_fun = bempp.api.GridFunction(
    bc_space, fun=tang, dual_space=snc_space)

E2 = - multitrace[1, 0]
E1 = multitrace[0, 1]
op = E1 * E2
rhs = E1 * grid_fun

sol, info = bempp.api.linalg.gmres(op, rhs, use_strong_form=True)


In [16]:
bc_space.flat_local_dof_count

2904

In [17]:
snc_space.flat_local_dof_count

1512

In [18]:
2904/1512

1.9206349206349207

In [19]:
multitrace.strong_form()

<252x252 BlockedDiscreteOperatorProduct with dtype=complex128>